# Homework 3 (100 points)

The goal of this homework is to practice techniques relating to SVD.

## Exercise 1 (65 points)

a) Fetch the "mnist_784" data and store is as a `.csv` (that way you don't have to fetch it every time - which takes about 30s). (4 points)

In [62]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
from os.path import exists

X, y = fetch_openml(name="mnist_784", version=1, return_X_y=True, as_frame=False)
data = pd.DataFrame({
        'Image': X.tolist(),
        'Label': y,
    })

In [66]:
if not exists('mnist.csv'):
    X, y = fetch_openml(name="mnist_784", version=1, return_X_y=True, as_frame=False)
    pd.DataFrame({
        # 'Image': [image for image in X],
        'Image': X.tolist(),
        'Label': y,
    }).sort_values(by=['Label']).to_csv('mnist.csv', index=False)

data = pd.read_csv('mnist.csv')

In [74]:
def clean_alt_list(list_):
    list_ = list_.replace(', ', '","')
    list_ = list_.replace('[', '["')
    list_ = list_.replace(']', '"]')
    return list_
data['Image'].apply(clean_alt_list)
print(type(data['Image'].iloc[0]))


<class 'str'>


b) Plot the singular value plot for a single example of the 0 digit (5 points)

In [57]:
zero = data[data['Label'] == 0].sample()

# print(type(zero))
zero = zero['Image'].iloc[0]
print(zero[0])
# u, s, v = np.linalg.svd()
# plt.imshow(zero.reshape(28, 28))

TypeError: Cannot interpret '<built-in function array>' as a data type

c) By setting some singular values to 0, plot the approximation of the 0 digit next to the original digit. (5 points)

d) Consider the entire dataset as a matrix. Perform SVD and store the dataset approximation in a new `.csv` file. Explain why / how you chose a particular rank. (10 points)

e) As in homework 2, using Kmeans on this new dataset, cluster the images from d) using 10 clusters and plot the centroid of each cluster. (10 points)

f) Repeat e) on the original dataset. Comment on any differences (or lack thereof) you observe. (8 points)

g) Compare the disagreement distance of the clustering obtained in e) to the true labels, to the disagreement distance of the clustering obtained in f) to the true labels. Comment briefly. (8 points)

In [ ]:
def disagreement_dist(P_labels, C_labels):
    return

h) Create a matrix that is the difference between the original dataset and the rank-10 approximation of the dataset. (10 points)

i) The largest (using euclidean distance from the origin) rows of the matrix could be considered anomalous data points. Briefly explain why. Plot the 10 images responsible for the 10 largest rows. (5 points)

## Exercise 2 (35 points)

a) Modify the code below to pick 4 categories of news articles that you think are minimally related (for example `sci.space` and `rec.sport.baseball`). (3 points)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer

categories = # your categories here
news_data = fetch_20newsgroups(subset='train', categories=categories)

b) Using the `SnowballStemmer`, stem the words in every article (10 points)

c) Use the `TfidfVectorizer` on the stemmed articles. Set `min_df` and `max_df` to reasonable numbers and briefly explain your reasoning. Store the resulting dataset into a `.csv` file. (7 points)

d) For rank k ranging from 1 to 25:

1. Reduce the dimensionality of the tfidf vectorized data using a dimension reduction technique discussed in class.
2. Apply Kmeans on the reduced dataset to create 4 clusters
3. Record the disagreement distance between the clustering in 2 and the article category

Then plot the recorded disagreement distance per rank. Comment briefly. (15 points)

In [ ]:
disagreement_distance = []
for k in range(1,25):
    dim_reduced_dataset = # your code here
    kmeans = KMeans(n_clusters=3, init='k-means++', max_iter=100, n_init=10, random_state=0)
    kmeans.fit_predict(dim_reduced_dataset)
    labelsk = kmeans.labels_
    disagreement_distance.append(disagreement_dist(labelsk, news_data.target))

plt.plot(range(1,25), disagreement_distance)
plt.ylabel('Disagreement')
plt.xlabel('Dimension')
plt.show()